In [39]:
import pandas
import random
import os
import json
import numpy as np
from datetime import datetime, timedelta

# How much data do we want to make?
ROWCOUNT = 1000

## Part 1

### Customer

In [40]:
custFileName = "nameList1000.txt"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + custFileName

first_names = []
last_names = []

with open(ROOT, 'r') as file:
    rawnames = file.read().split('\n')

for name in rawnames:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

In [41]:
'''
--
-- given list of 1st and 2nd names
-- randomize them and duplicate them randomly for repeat customers
--
'''

# Create a list to hold all names
names = []

# Generate unique names (50% of 4000 = 2000 unique names)
for index in range(2000):
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    names.append(name)

# Generate repeated names (remaining 50% will be repeats 5-10 times)
for index in range(ROWCOUNT):  # Need 1000 more distinct names, each repeating 5-10 times
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    repeat_count = random.randint(5, 10)  # Repeat each name 5 to 10 times
    names.extend([name] * repeat_count)

# Shuffle the final list to mix unique and repeated names
random.shuffle(names)

# Ensure the list is exactly 1000 names
names = names[:ROWCOUNT]

# Convert the list to a single line with commas separating each name
# result = ", ".join(names)

In [42]:
'''
--
-- DF to pandas
--
'''
customerDict = {}
first_names = []
last_names = []

for name in names:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

customerDict["First"] = first_names
customerDict["Last"] = last_names

customerDF = pandas.DataFrame.from_dict(data=customerDict)
customerDF


,First,Last
0,Aria,Davis
1,Charlotte,Foster
2,Nora,Parker
3,Anna,Ross
4,Bella,Ortiz
...,...,...
995,Eleanor,Chavez
996,Gianna,Watson
997,Charlotte,Gray
998,Penelope,Rivera


### Store

In [43]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

storeDict = {}

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Store"]

listAdd = []
for dict in data:
    listAdd.append(data[dict])

    
storeDf = pandas.DataFrame.from_records(listAdd)
storeDf

,Name,Address,City,Texas,Zip Code
0,South Austin Drive,102 South Austin Drive,Austin,Texas,72034
1,Western Drive,111 Western Drive,Houston,Texas,99345
2,North Lake,880 North Lake Drive,Dallas,Texas,14783


### Employee

In [44]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Employee"]

# get each stores employees
stores = []
for store in data:
    stores.append(data[store])


In [45]:
'''
{'one': {'EmployeeID': '1', 'ManagerID': 'Null', 'FirstName': 'Tom', 'LastName': 'Bates', 'StoreID': '1'}, 'two': {
'''

# Transpose json into wanted shape
df = pandas.DataFrame(stores[0]).T
# remove the unwanted index
stores.pop(0)

# merge wanted df into new df
for data in stores:
    dfNew = pandas.DataFrame(data).T
    df = pandas.concat([df,dfNew])

Employeedf = df
Employeedf.reset_index(drop=True, inplace=True)


### Product

In [46]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Product"]

# get each stores employees
product = []
for products in data:
    product.append(data[products])

productDF = pandas.DataFrame.from_records(product).T

productDF = productDF.rename(columns={0: 'Product', 1: 'Price'})

productDF['productID'] = productDF.reset_index().index + 1


### Full Table DF - Store

In [47]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(storeDf))
# random_counts

while storeDf.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        
        # checker to prevent concatenating an empty list,
        if count > 0:
        
            # storeDf.concat([storeDf.loc[[i]] * count])
            row_to_duplicate = storeDf.loc[[row]]

            # Duplicate the row 2 times
            duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)

            # Create a new DataFrame with the duplicated rows
            storeDf = pandas.concat([storeDf, duplicated_row], ignore_index=True)

# limit to 1000 rows
storeDf = storeDf.iloc[:ROWCOUNT]

### Full Table DF - Employee

In [48]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(Employeedf))
random_counts


array([10, 19, 12,  9,  8,  6,  4, 17,  2,  9,  3,  5,  0,  5])

In [49]:

while Employeedf.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        # storeDf.concat([storeDf.loc[[i]] * count])
        
        # checker to prevent concatenating an empty list,
        if count > 0:
                
            row_to_duplicate = Employeedf.loc[[row]]

            # Duplicate the row 2 times
            duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)

            # Create a new DataFrame with the duplicated rows
            Employeedf = pandas.concat([Employeedf, duplicated_row], ignore_index=True)

# limit to 1000 rows
Employeedf = Employeedf.iloc[:ROWCOUNT]

# Full table Df - Product

In [50]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(productDF))
random_counts


while productDF.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        
        # checker to prevent concatenating an empty list,
        if count > 0:
                
            # storeDf.concat([storeDf.loc[[i]] * count])
            row_to_duplicate = productDF.loc[[row]]
                
            # Duplicate the row
            duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)
            
            # Create a new DataFrame with the duplicated rows
            productDF = pandas.concat([productDF, duplicated_row], ignore_index=True)

# limit to 1000 rows
productDF = productDF.iloc[:ROWCOUNT]

## Concat ALl Tables into 1 table

In [51]:
fullTableDF = pandas.concat([customerDF,storeDf], axis=1)
fullTableDF = pandas.concat([fullTableDF,Employeedf], axis=1)
fullTableDF = pandas.concat([fullTableDF,productDF], axis=1)

In [52]:
# Set today's date
date = datetime.today().date()

# Define the time ranges with higher frequencies
morning_peak_start = 7.5  # 7:30am
morning_peak_end = 9
afternoon_peak_start = 15.5  # 3:30pm
afternoon_peak_end = 17

times = []

for _ in range(ROWCOUNT):
    # 40% chance of morning peak
    if np.random.rand() < 0.4:  
        hour = np.random.uniform(morning_peak_start, morning_peak_end)
    # 30% chance of afternoon peak
    elif np.random.rand() < 0.7:  
        hour = np.random.uniform(afternoon_peak_start, afternoon_peak_end)
    # 40% chance of random time between 6am and 9pm
    else:  
        hour = np.random.uniform(6, 21)

    minute = np.random.uniform(0, 60)
    time = datetime.combine(date, datetime.min.time()) + timedelta(hours=int(hour), minutes=int(minute))
    times.append(time)

# Create the pandas DataFrame
df = pd.DataFrame({'date': [date] * len(times), 'time': times})
dfDateTime = df.drop('date', axis=1)

NameError: name 'pd' is not defined

In [27]:
fullTableDF = pandas.concat([fullTableDF,dfDateTime], axis=1)

In [28]:
fullTableDF

,First,Last,Name,Address,City,Texas,Zip Code,EmployeeID,ManagerID,FirstName,LastName,StoreID,Product,Price,productID,time
0,Luna,Parker,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,1,Null,Tom,Bates,1,Hot Cocoa,4.5,1,2024-10-15 08:26:00
1,Gianna,Evans,Western Drive,111 Western Drive,Houston,Texas,99345,2,1,Alex,Dune,1,Espresso,3,2,2024-10-15 07:59:00
2,Sarah,Parker,North Lake,880 North Lake Drive,Dallas,Texas,14783,3,1,Frank,Watts,1,Ristretto,3.5,3,2024-10-15 15:36:00
3,Natalia,Thompson,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,4,1,Daniel,Farmer,1,Americano,4,4,2024-10-15 15:29:00
4,Bella,Mitchell,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,5,Null,Toby,Harts,2,Latte,5.5,5,2024-10-15 16:12:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Riley,Smith,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Cafe au Lait,4,19,2024-10-15 15:54:00
996,Chloe,Cruz,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Cafe au Lait,4,19,2024-10-15 11:11:00
997,Autumn,Walker,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Hot Cocoa,4.5,1,2024-10-15 08:22:00
998,Naomi,Kelly,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,10,Null,Arthur,Alan,3,Hot Cocoa,4.5,1,2024-10-15 15:03:00


## GCP Push

In [29]:
fullTableDF.columns = ['CustomerFirstName', 'CustomerLastName', 'StoreName', 'StoreAddress','City','State','ZipCode','EmployeeId','ManagerId','EmployeeFirstName','EmployeeLastName','StoreId','ProductName','Price','ProductId','SaleDate']

In [53]:
fullTableDF

,First,Last,Name,Address,City,Texas,Zip Code,EmployeeID,ManagerID,FirstName,LastName,StoreID,Product,Price,productID
0,Aria,Davis,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,1,Null,Tom,Bates,1,Hot Cocoa,4.5,1
1,Charlotte,Foster,Western Drive,111 Western Drive,Houston,Texas,99345,2,1,Alex,Dune,1,Espresso,3,2
2,Nora,Parker,North Lake,880 North Lake Drive,Dallas,Texas,14783,3,1,Frank,Watts,1,Ristretto,3.5,3
3,Anna,Ross,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,4,1,Daniel,Farmer,1,Americano,4,4
4,Bella,Ortiz,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,5,Null,Toby,Harts,2,Latte,5.5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Eleanor,Chavez,Western Drive,111 Western Drive,Houston,Texas,99345,1,Null,Tom,Bates,1,Macchiato,4.25,7
996,Gianna,Watson,Western Drive,111 Western Drive,Houston,Texas,99345,1,Null,Tom,Bates,1,Macchiato,4.25,7
997,Charlotte,Gray,Western Drive,111 Western Drive,Houston,Texas,99345,1,Null,Tom,Bates,1,Macchiato,4.25,7
998,Penelope,Rivera,Western Drive,111 Western Drive,Houston,Texas,99345,1,Null,Tom,Bates,1,Macchiato,4.25,7



## How to connect
- How to: https://cloud.google.com/blog/topics/developers-practitioners/how-connect-cloud-sql-using-python-easy-way
- how to: https://cloud.google.com/sql/docs/mysql/samples/cloud-sql-mysql-sqlalchemy-connect-connector
-
- ./google-cloud-sdk/bin/gcloud init
- start again Y or N 
- gcloud auth login 
- copy link and login 
- gcloud config set project PROJECT_ID
- gcloud auth application-default login
-
-


In [ ]:
# How to: https://cloud.google.com/blog/topics/developers-practitioners/how-connect-cloud-sql-using-python-easy-way
# how to: https://cloud.google.com/sql/docs/mysql/samples/cloud-sql-mysql-sqlalchemy-connect-connector

gcloud init
gcloud auth login 
gcloud config set project PROJECT_ID
gcloud auth application-default login

In [57]:

'''
-- How to: https://cloud.google.com/blog/topics/developers-practitioners/how-connect-cloud-sql-using-python-easy-way
-- how to: https://cloud.google.com/sql/docs/mysql/samples/cloud-sql-mysql-sqlalchemy-connect-connector
--
-- ./google-cloud-sdk/bin/gcloud init
-- start again Y or N 
-- gcloud auth login 
-- copy link and login 
-- gcloud config set project PROJECT_ID
-- gcloud auth application-default login
--
--
'''


from google.cloud.sql.connector import Connector

import sqlalchemy
import pymysql
import os 
import json

custFileName = "keysData.json"
ROOT = os.path.dirname(os.getcwd()) + '/resource/' + custFileName

with open(ROOT, 'r') as file:
    jsonData = json.load(file)
    
    project_id = jsonData["GCP"]["project_id"]
    region = jsonData["GCP"]["region"]
    instance_name = jsonData["GCP"]["instance_name"]
    DB_USER = jsonData["GCP"]["DB_USER"]
    DB_PASS = jsonData["GCP"]["DB_PASS"]
    DB_NAME = jsonData["GCP"]["DB_NAME"]
    TABLE = jsonData["GCP"]["TABLE"]
    INSTANCE_CONNECTION_NAME = jsonData["GCP"]["INSTANCE_CONNECTION_NAME"]
    IP = jsonData["GCP"]["IP"]
    TableRaw = jsonData["GCP"]["TABLERAW"]


print(" -- -- -- ")    
print("items:")

print(INSTANCE_CONNECTION_NAME)
print(DB_USER)
print(DB_PASS)
print(DB_NAME)
print(" -- -- -- ")

# initialize Connector object
connector = Connector()
print("connector ^ done ")


# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

# insert statement
# insert_stmt = sqlalchemy.text( "INSERT INTO my_table (id, title) VALUES (:id, :title)",)


# Table ['CustomerFirstName', 'CustomerLastName', 'StoreName', 'StoreAddress','City','State','ZipCode','EmployeeId','ManagerId','EmployeeFirstName','EmployeeLastName','StoreId','ProductName','Price','ProductId','SaleDate']
# CustomerFirstName,CustomerLastName, StoreName, StoreAddress,City,State,ZipCode,EmployeeId,ManagerId,EmployeeFirstName,EmployeeLastName,StoreId,ProductName,Price,ProductId,SaleDate

with pool.connect() as db_conn:
    
    
    ''' 
    #Push Df with to_sql 
    fullTableDF.to_sql('tableOne', con=pool, if_exists='append', index=False) 
    '''

    try:
        
        # Iterate through each row of the DataFrame
        for index, row in fullTableDF.iterrows():
        
            # Create an insert statement
            insert_statement = f'INSERT INTO {TableRaw} (CustomerFirstName,CustomerLastName, StoreName, StoreAddress, City,State,ZipCode,EmployeeId,ManagerId,EmployeeFirstName,EmployeeLastName,StoreId,ProductName,Price,ProductId,SaleDate ) VALUES (:CustomerFirstName, :CustomerLastName, :StoreName, :StoreAddress, :City, :State, :ZipCode, :EmployeeId, :ManagerId, :EmployeeFirstName, :EmployeeLastName, :StoreId, :ProductName, :Price, :ProductId, :SaleDate )'  

            # Execute the insert statement with row data
            db_conn.execute(insert_statement, 
                column1=row['CustomerFirstName'], 
                column2=row['CustomerLastName'],
                column3=row['StoreName'],
                column4=row['StoreAddress'],
                column5=row['City'],
                column6=row['State'],
                column7=row['ZipCode'],
                column8=row['EmployeeId'],
                column9=row['ManagerId'],
                column10=row['EmployeeFirstName'],
                column11=row['EmployeeLastName'],
                column12=row['ProductName'],
                column13=row['Price'],
                column14=row['ProductId'],
                column15=row['SaleDate']
                )
            
            print(f"All rows inserted into {TableRaw}.")

    except Exception as e:
        print(f"An error occured when pushing data:::\n{e}")
        
    
    # query database
    # result = db_conn.execute(sqlalchemy.text(f"SELECT * from {TABLE}")).fetchall()


    # Do something with the results
    # for row in result:
    #     print(row)

print("done")



 -- -- -- 
items:
micro-flight-436817-p7:us-central1:coffee123
coffee123
[]J3Lg;_B24$
Coffee
 -- -- -- 
connector ^ done 


TLSv1.3 is not supported with your version of OpenSSL (LibreSSL 2.8.3), falling back to TLSv1.2
Upgrade your OpenSSL version to 1.1.1 for TLSv1.3 support.


An error occured when pushing data:::
'CustomerFirstName'
done


In [55]:

insert_stmt = sqlalchemy.text(f"INSERT INTO {TableRaw} (CustomerFirstName, CustomerLastName, StoreName, StoreAddress,City,State,ZipCode,EmployeeId,ManagerId,EmployeeFirstName,EmployeeLastName,StoreId,ProductName,Price,ProductId,SaleDate) VALUES (:CustomerFirstName,:CustomerLastName,: StoreName,: StoreAddress,:City,:State,:ZipCode,:EmployeeId,:ManagerId,:EmployeeFirstName,:EmployeeLastName,:StoreId,:ProductName,:Price,:ProductId,:SaleDate)",)

with pool.connect() as db_conn:
    
    # insert into database
    # db_conn.execute(insert_stmt, parameters={"CustomerFirstName": "book1", "CustomerLastName": "Book One"})
    db_conn.execute(insert_stmt, parameters= fullTableDF)

    # commit transaction
    # db_conn.commit()


ArgumentError: mapping or list expected for parameters